In [2]:
import time
import os
import seaborn as sns
import numpy as np
import pandas as pd
import sys
%load_ext autoreload
%autoreload 2
from mimas.tools.spectral_file.extract_ms1_feature import process_mzml_file, extract_ms2_spectra, find_features
import logging
from toolsets.spectra_operations import entropy_similarity_default
from toolsets.std_list_prep import complete_adducts, complete_formula
# from toolsets.API_gets import complete_smiles
from toolsets.search import string_search, num_search
import toolsets.spectra_operations as so
from tqdm import tqdm
from toolsets.features_by_alphapept import find_features_alphapept
import matplotlib.pyplot as plt
import seaborn as sns
EAD_source = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD'
EAD_bio_source = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD/biological_samples/'
UVPD_source = "/Users/fanzhoukong/Documents/GitHub/Libgen_data/UVPD"
figure_source = "/Users/fanzhoukong/Documents/GitHub/Libgen_data/manuscript"
ead_source_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/EAD'
uvpd_source_dir = "/Users/fanzhoukong/Documents/GitHub/Libgen_data/UVPD"

uvpd_lib = pd.read_csv(os.path.join(uvpd_source_dir, 'UVPD_library_cla.csv'))
hcd_lib = pd.read_csv(os.path.join(uvpd_source_dir, 'HCD_library_cla.csv'))
ead_lib = pd.read_csv(os.path.join(ead_source_dir, 'EAD_library_cla.csv'))
# ead_pfp_lib =pd.read_csv(os.path.join(ead_source_dir, 'data_pfp_lib_final.csv'))
# ead_hilic_lib =pd.read_csv(os.path.join(ead_source_dir, 'data_hilic_lib_final.csv'))
# ead_lib = pd.concat([ead_pfp_lib, ead_hilic_lib], axis=0, ignore_index=True)

In [3]:
ead_replib = pd.read_csv(os.path.join(ead_source_dir, 'matched_mapped_with_duplicates_all_denoised_post.csv'))

In [11]:
from toolsets.file_io import read_msp_files

In [20]:
hilic_bio = read_msp_files(os.path.join(EAD_bio_source, 'HILIC_pos.msp'))
pfp_bio = read_msp_files(os.path.join(EAD_bio_source, 'PFP_pos.msp'))
bio_all = pd.concat([hilic_bio, pfp_bio], axis=0)
bio_all.to_csv(os.path.join(EAD_source, 'bio_all.csv'), index = False)

cleaning incoming spectrum


100%|██████████| 3333/3333 [04:39<00:00, 11.90it/s]


cleaning incoming spectrum


100%|██████████| 2431/2431 [02:32<00:00, 15.91it/s]


In [5]:
from toolsets.msms_search import entropy_search_identity

i am msms_search!!!!!


In [67]:
# searched_rip = pd.DataFrame()
# bio_peaks_rip = []
# rt_rip = []
# for index, row in tqdm(bio_all.iterrows(), total=len(bio_all)):
#     search_result =entropy_search_identity(row['peaks'], row['PRECURSORMZ'], ead_replib, return_type='max')
#     if (search_result) is not np.NAN:
#         bio_peaks_rip.append(row['peaks'])
#         rt_rip.append(row['RETENTIONTIME'])
#         searched_rip= searched_rip.append(search_result)


100%|██████████| 5764/5764 [05:47<00:00, 16.59it/s]  


In [3]:
bio_all = pd.read_csv(os.path.join(EAD_source, 'bio_all.csv'))

In [6]:
searched = pd.DataFrame()
bio_peaks = []
rt = []
for index, row in tqdm(bio_all.iterrows(), total=len(bio_all)):
    search_result =entropy_search_identity(row['peaks'], row['PRECURSORMZ'], ead_lib, return_type='max')
    if (search_result) is not np.NAN:
        bio_peaks.append(row['peaks'])
        searched= searched.append(search_result)
        rt.append(row['RETENTIONTIME'])

100%|██████████| 5764/5764 [04:09<00:00, 23.13it/s] 


In [8]:
searched['matched_entropy'].describe()

count    89.000000
mean      0.824958
std       0.050258
min       0.750015
25%       0.785599
50%       0.822591
75%       0.855771
max       0.952315
Name: matched_entropy, dtype: float64

In [75]:
std_list_hilic=pd.read_csv(os.path.join(ead_source_dir, 'std_list_hilic_cleaned.csv'))
std_list_pfp=pd.read_csv(os.path.join(ead_source_dir, 'std_list_pfp_cleaned.csv'))

In [76]:
std_list_ead = pd.concat([std_list_hilic, std_list_pfp], axis=0)

In [78]:
len(std_list_ead['inchikey'].unique())

2951

In [79]:
std_list_uvpd = pd.read_csv(os.path.join(uvpd_source_dir, 'std_list_pos_ready.csv'))

In [81]:
len(std_list_uvpd['inchikey'].unique())

2007

In [73]:
len(ead_lib['reference_inchikey'].unique())

1813